In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from google.cloud import storage
from fastai.vision import *
from fastai.metrics import error_rate

In [ ]:
bs = 64

### Regex for extracting labels : ([^/]+)(?=_)

<pre>
[^/] : Any character that is not a / <br>
/   : Matches the character / literally
\+  : Matches between one and unlimited times, as many times as possible, giving back as needed (greedy) <br>
\(?=_)  : Positive Lookahead <br>
    _ : matches the character _ literally (case sensitive) <br>
</pre>

In [ ]:
pat = r'([^/]+)(?=_)'

In [ ]:
fnames = list(filter(lambda x: x!=".DS_Store" , get_image_files("data/images")))
fnames[:5]

In [ ]:
np.random.seed(2)
data = ImageDataBunch.from_name_re('data/images', fnames, pat, ds_tfms=get_transforms(), size=224, bs=bs).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
print(data.classes)
len(data.classes),data.c

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

In [ ]:
learn.model

In [ ]:
learn.fit_one_cycle(4)

In [ ]:
learn.save('stage-1')

### Results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
doc(interp.plot_top_losses)

In [ ]:
interp.most_confused(min_val=2)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

### Unfreezing, fine-tuning, and learning rates

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(1)

In [ ]:
learn.load('stage-1')

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, max_lr=slice(1e-6,1e-5))